Prompt:

請生成一個完整的 Python 程式碼，該程式會執行以下步驟：

從給定的 Google Drive 連結下載壓縮檔案，解壓後載入資料，包括 train.csv 和 test.csv。
執行資料預處理，包括：
將 Age、Fare 和 Cabin 分組。
將 Name 分解為姓氏和稱謂。
移除與模型無關的特徵，如 Ticket 和 Embarked。
使用 PyCaret 建立分類模型，對資料進行建模和性能比較，選擇最佳模型。
使用 Optuna 進行超參數優化，並基於最佳參數重新訓練模型。
輸出：
驗證資料的準確率。
測試資料的預測結果檔案，格式為兩欄：PassengerId 和 Survived，儲存為 submission.csv。
在程式中，應包括清晰的註解來說明每個步驟。
提示補充：

資料集的目標欄位是 Survived。
驗證準確率需計算並輸出到控制台。
使用 PyCaret 的堆疊模型進行建模，且使用 Optuna 優化堆疊模型的超參數。
測試資料的預測結果不含真實標籤，直接生成 CSV 文件即可。

In [1]:
!pip install pycaret scikit-learn
!pip install optuna

  Using cached optuna-4.1.0-py3-none-any.whl.metadata (16 kB)
  Using cached alembic-1.14.0-py3-none-any.whl.metadata (7.4 kB)
  Using cached colorlog-6.9.0-py3-none-any.whl.metadata (10 kB)
Using cached optuna-4.1.0-py3-none-any.whl (364 kB)
Using cached alembic-1.14.0-py3-none-any.whl (233 kB)
Using cached colorlog-6.9.0-py3-none-any.whl (11 kB)


In [2]:
import os
import zipfile
import requests
import pandas as pd
from sklearn.model_selection import train_test_split
from pycaret.classification import *
from sklearn.metrics import accuracy_score
import optuna

# 1. 資料下載與解壓
url = "https://drive.google.com/uc?id=1ZfxPkqBfZa4SFVPbwmyghbLiiKNUqfCj"
zip_file_name = "data.zip"
extracted_folder = "data"

def download_file(url, file_name):
    response = requests.get(url, stream=True)
    with open(file_name, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)

if not os.path.exists(zip_file_name):
    download_file(url, zip_file_name)

if not os.path.exists(extracted_folder):
    with zipfile.ZipFile(zip_file_name, "r") as zip_ref:
        zip_ref.extractall(extracted_folder)

# 2. 資料讀取與預處理
data_train_path = os.path.join(extracted_folder, "train.csv")
data_test_path = os.path.join(extracted_folder, "test.csv")

data_train = pd.read_csv(data_train_path)
data_test = pd.read_csv(data_test_path)

# 2.1 Feature Engineering
def simplify_ages(df):
    df.Age = df.Age.fillna(-0.5)
    bins = [-1, 0, 5, 12, 18, 25, 35, 60, 120]
    group_names = ['Unknown', 'Baby', 'Child', 'Teenager', 'Student', 'Young Adult', 'Adult', 'Senior']
    df.Age = pd.cut(df.Age, bins, labels=group_names)
    return df

def simplify_cabins(df):
    df.Cabin = df.Cabin.fillna('N')
    df.Cabin = df.Cabin.map(lambda x: x[0])
    return df

def simplify_fares(df):
    df.Fare = df.Fare.fillna(-0.5)
    bins = [-1, 0, 8, 15, 31, 1000]
    group_names = ['Unknown', '1_quartile', '2_quartile', '3_quartile', '4_quartile']
    df.Fare = pd.cut(df.Fare, bins, labels=group_names)
    return df

def format_name(df):
    df['Lname'] = df.Name.map(lambda x: x.split(' ')[0])
    df['NamePrefix'] = df.Name.map(lambda x: x.split(' ')[1])
    return df

def drop_features(df):
    return df.drop(['Ticket', 'Name', 'Embarked'], axis=1)

def transform_features(df):
    df = simplify_ages(df)
    df = simplify_cabins(df)
    df = simplify_fares(df)
    df = format_name(df)
    df = drop_features(df)
    return df

data_train = transform_features(data_train)
data_test = transform_features(data_test)

# 分割訓練和測試數據
train_data, validation_data = train_test_split(data_train.drop(['PassengerId'], axis=1),
                                               random_state=100, train_size=0.8)

# 3. 使用 PyCaret 進行模型選擇與訓練
clf1 = setup(data=train_data,
             target='Survived',
             categorical_features=['Pclass', 'Sex', 'Age', 'Fare', 'Cabin', 'Lname', 'NamePrefix'],
             session_id=42)

# 比較模型
top_models = compare_models(n_select=5, fold=5)

# 建立堆疊模型（初步）
stacker = stack_models(estimator_list=top_models, meta_model=create_model('lr'))

# 儲存初步模型
save_model(stacker, 'stacker_initial')


,Description,Value
0,Session id,42
1,Target,Survived
2,Target type,Binary
3,Original data shape,"(712, 10)"
4,Transformed data shape,"(712, 52)"
5,Transformed train set shape,"(498, 52)"
6,Transformed test set shape,"(214, 52)"
7,Numeric features,2
8,Categorical features,7
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.8094,0.8477,0.6950,0.7752,0.7295,0.5837,0.5884,2.6600
ridge,Ridge Classifier,0.8033,0.8465,0.6469,0.7930,0.7117,0.5647,0.5721,0.2140
et,Extra Trees Classifier,0.7772,0.8017,0.5881,0.7661,0.6602,0.5002,0.5128,0.2840
rf,Random Forest Classifier,0.7731,0.8136,0.5991,0.7509,0.6637,0.4962,0.5053,0.5040
knn,K Neighbors Classifier,0.7711,0.8250,0.5929,0.7428,0.6568,0.4893,0.4978,0.2740
xgboost,Extreme Gradient Boosting,0.7288,0.7534,0.4963,0.6865,0.5588,0.3799,0.3950,0.2120
ada,Ada Boost Classifier,0.7249,0.7275,0.4912,0.6808,0.5520,0.3700,0.3880,0.2340
gbc,Gradient Boosting Classifier,0.7208,0.7023,0.4805,0.6676,0.5396,0.3585,0.3743,0.2500
lda,Linear Discriminant Analysis,0.6927,0.7520,0.2989,0.7023,0.4166,0.2563,0.2981,0.1380
svm,SVM - Linear Kernel,0.6668,0.7501,0.6525,0.6107,0.5860,0.3302,0.3481,0.1440


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8000,0.7656,0.7778,0.7000,0.7368,0.5763,0.5784
1,0.8600,0.9083,0.7895,0.8333,0.8108,0.6998,0.7005
2,0.7800,0.8684,0.7368,0.7000,0.7179,0.5378,0.5383
3,0.7800,0.8065,0.5789,0.7857,0.6667,0.5081,0.5212
4,0.8200,0.8947,0.7368,0.7778,0.7568,0.6141,0.6146
5,0.8000,0.8387,0.7895,0.7143,0.7500,0.5840,0.5861
6,0.8800,0.9219,0.7895,0.8824,0.8333,0.7400,0.7428
7,0.8200,0.7980,0.6316,0.8571,0.7273,0.5975,0.6130
8,0.8776,0.8772,0.7778,0.8750,0.8235,0.7303,0.7332


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7800,0.7240,0.6111,0.7333,0.6667,0.5045,0.5092
1,0.8600,0.9117,0.6316,1.0000,0.7742,0.6801,0.7178
2,0.7600,0.8362,0.6842,0.6842,0.6842,0.4907,0.4907
3,0.7400,0.7861,0.5263,0.7143,0.6061,0.4186,0.4295
4,0.7600,0.8090,0.6842,0.6842,0.6842,0.4907,0.4907
5,0.7800,0.8370,0.6316,0.7500,0.6857,0.5184,0.5229
6,0.8400,0.9100,0.7368,0.8235,0.7778,0.6534,0.6558
7,0.7800,0.7776,0.5263,0.8333,0.6452,0.4973,0.5248
8,0.7959,0.8163,0.6667,0.7500,0.7059,0.5505,0.5527


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None, include=['SibSp', 'Parch'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=False,
                                                               missing_values=nan,
                                                               strategy='mean'))),
                 ('categorical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['Pclass', 'Sex', 'Age', 'Fare...
                                                                       p=2,
                                                                       weights='uniform'))],
        

In [3]:

# 4. 超參數優化 - 使用 Optuna
def objective(trial):
    params = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
    }
    model = create_model('xgboost', fold=5, **params)
    stacker_optimized = stack_models(estimator_list=top_models, meta_model=model)
    final_pred = predict_model(stacker_optimized, data=validation_data)
    score = accuracy_score(validation_data['Survived'], final_pred['prediction_label'])
    return score

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)


[I 2024-11-27 09:48:11,902] A new study created in memory with name: no-name-a60e2f76-180f-446b-ae6b-3427d868ef54


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7600,0.8078,0.4595,0.8095,0.5862,0.4348,0.4694
1,0.7900,0.7927,0.6053,0.7931,0.6866,0.5329,0.5439
2,0.7700,0.8410,0.6316,0.7273,0.6761,0.4991,0.5021
3,0.6869,0.8309,0.2703,0.7143,0.3922,0.2352,0.2857
4,0.7475,0.8413,0.5135,0.7308,0.6032,0.4262,0.4404
Mean,0.7509,0.8227,0.4960,0.7550,0.5888,0.4256,0.4483
Std,0.0349,0.0194,0.1288,0.0386,0.1059,0.1032,0.0883


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8000,0.7222,0.6667,0.7500,0.7059,0.5552,0.5574
1,0.8400,0.9244,0.5789,1.0000,0.7333,0.6303,0.6784
2,0.7800,0.8379,0.7368,0.7000,0.7179,0.5378,0.5383
3,0.7400,0.7980,0.5263,0.7143,0.6061,0.4186,0.4295
4,0.7600,0.8277,0.6842,0.6842,0.6842,0.4907,0.4907
5,0.7800,0.8633,0.6316,0.7500,0.6857,0.5184,0.5229
6,0.8600,0.8913,0.7895,0.8333,0.8108,0.6998,0.7005
7,0.7800,0.7767,0.5263,0.8333,0.6452,0.4973,0.5248
8,0.7755,0.8584,0.5556,0.7692,0.6452,0.4872,0.5010


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7400,0.7335,0.3889,0.7778,0.5185,0.3665,0.4078
1,0.7600,0.9244,0.3684,1.0000,0.5385,0.4197,0.5154
2,0.7800,0.8336,0.6316,0.7500,0.6857,0.5184,0.5229
3,0.7800,0.7861,0.4211,1.0000,0.5926,0.4742,0.5575
4,0.7800,0.8081,0.6316,0.7500,0.6857,0.5184,0.5229
5,0.8000,0.8430,0.6316,0.8000,0.7059,0.5575,0.5665
6,0.8400,0.8981,0.6842,0.8667,0.7647,0.6460,0.6564
7,0.7800,0.7895,0.5263,0.8333,0.6452,0.4973,0.5248
8,0.7959,0.8566,0.5556,0.8333,0.6667,0.5279,0.5505


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Stacking Classifier,0.7598,0.8452,0.5467,0.8200,0.6560,0.4826,0.5060


[I 2024-11-27 10:11:01,205] Trial 49 finished with value: 0.7597765363128491 and parameters: {'learning_rate': 0.0024198726619997363, 'max_depth': 5, 'n_estimators': 178, 'subsample': 0.516400261921609, 'colsample_bytree': 0.5495616780532524}. Best is trial 13 with value: 0.8212290502793296.


In [4]:

# 最優參數
best_params = study.best_params
print("Best parameters found by Optuna:")
print(best_params)

# 使用最優參數訓練最終模型
final_model = create_model('xgboost', fold=5, **best_params)
final_stacker = stack_models(estimator_list=top_models, meta_model=final_model)

# 評估模型性能
final_pred = predict_model(final_stacker, data=data_test)


Best parameters found by Optuna:
{'learning_rate': 0.012557543139431113, 'max_depth': 4, 'n_estimators': 169, 'subsample': 0.5110495844017058, 'colsample_bytree': 0.9095623979615368}


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6700,0.7765,0.2162,0.6667,0.3265,0.1775,0.2269
1,0.7000,0.7772,0.3421,0.7222,0.4643,0.2911,0.3303
2,0.7400,0.7958,0.5789,0.6875,0.6286,0.4308,0.4346
3,0.6364,0.7960,0.1351,0.5556,0.2174,0.0833,0.1188
4,0.7677,0.7661,0.6757,0.6944,0.6849,0.5010,0.5011
Mean,0.7028,0.7823,0.3896,0.6653,0.4643,0.2967,0.3224
Std,0.0471,0.0118,0.2072,0.0577,0.1764,0.1546,0.1379


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7600,0.7170,0.5556,0.7143,0.6250,0.4526,0.4603
1,0.8600,0.9482,0.6316,1.0000,0.7742,0.6801,0.7178
2,0.7400,0.7835,0.6316,0.6667,0.6486,0.4425,0.4429
3,0.7600,0.7878,0.5789,0.7333,0.6471,0.4690,0.4766
4,0.7600,0.7691,0.6842,0.6842,0.6842,0.4907,0.4907
5,0.7800,0.8455,0.6316,0.7500,0.6857,0.5184,0.5229
6,0.8400,0.8761,0.7368,0.8235,0.7778,0.6534,0.6558
7,0.7800,0.7360,0.5263,0.8333,0.6452,0.4973,0.5248
8,0.7755,0.7966,0.5556,0.7692,0.6452,0.4872,0.5010


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [5]:
# 確保驗證數據有真實標籤進行評估
final_pred = predict_model(final_stacker, data=validation_data)
print("Validation Accuracy:", accuracy_score(validation_data['Survived'], final_pred['prediction_label']))

# 測試數據的預測（沒有真實值）
final_test_pred = predict_model(final_stacker, data=data_test)
print(final_test_pred.head())  # 僅輸出預測


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Stacking Classifier,0.8212,0.8097,0.7200,0.8308,0.7714,0.6259,0.6302


Validation Accuracy: 0.8212290502793296


   PassengerId  Pclass     Sex          Age  SibSp  Parch        Fare Cabin  \
0          892       3    male  Young Adult      0      0  1_quartile     N   
1          893       3  female        Adult      1      0  1_quartile     N   
2          894       2    male       Senior      0      0  2_quartile     N   
3          895       3    male  Young Adult      0      0  2_quartile     N   
4          896       3  female      Student      1      1  2_quartile     N   

       Lname NamePrefix  prediction_label  prediction_score  
0     Kelly,        Mr.                 0            0.9271  
1    Wilkes,       Mrs.                 1            0.5386  
2     Myles,        Mr.                 0            0.8849  
3      Wirz,        Mr.                 0            0.9479  
4  Hirvonen,       Mrs.                 1            0.6018  
